# **Computer Vision 이상치 탐지 알고리즘 경진대회**

start : 220401




# Import module

In [ ]:
import matplotlib.pyplot as plt
import torch
import os
from tqdm import tqdm
from glob import glob
import cv2
import numpy as np
import pandas as pd
import PIL 
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from PIL import Image

# Set drive

In [ ]:
from google.colab import drive
drive.mount._DEBUG = True
drive.mount('/content/drive', force_remount=True)

In [ ]:
os.chdir('./drive/MyDrive/이상치/')

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
    
device

# Load data

In [ ]:
# %%time
# !unzip "./open.zip" -d "/content/drive/MyDrive/Dacon/Computer Vision 이상치 탐지 알고리즘 경진대회/data"

In [ ]:
# %%time
# !unzip "./data/train.zip" -d "/content/drive/MyDrive/이상치/data"

In [ ]:
%%time
!unzip "./data/test.zip" -d "/content/drive/MyDrive/이상치/data"

In [ ]:
train_x = sorted(glob('./data/train/*.png'))
test = sorted(glob('./data/test/*.png'))
train_csv = pd.read_csv("./data/train_df.csv")
train_label = train_csv["label"]

# Dataset

In [ ]:
label_unique = sorted(np.unique(train_label))

label_unique_dir = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_Y = [label_unique_dir[k] for k in train_label]

In [ ]:
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, x_dir,transform=None):
        super().__init__()
        self.transforms = transform
        self.x_img = x_dir 

    def __len__(self):
        return len(self.x_img)

    def __getitem__(self, idx):
        x_img = self.x_img[idx]

        x_img = cv2.imread(x_img)
        x_img = cv2.cvtColor(x_img, cv2.COLOR_BGR2RGB)

        if self.transforms:
            augmented = self.transforms(image=x_img)
            x_img = augmented['image']

        return x_img

In [ ]:
! pip install albumentations==0.4.6

In [ ]:
import albumentations
import albumentations.pytorch

In [ ]:
def transform_inputsize(inputsize):
  return albumentations.Compose([
      albumentations.Resize(inputsize, inputsize),
      albumentations.augmentations.transforms.Normalize(mean=(0.5,), std=(0.5,), p=1.0),
      albumentations.pytorch.transforms.ToTensorV2(p=1.0)
      ])

In [ ]:
batch_size=32
test_dataset224 = TestDataset(test,transform_inputsize(224))
test_loader224 = torch.utils.data.DataLoader(test_dataset224, shuffle=False, batch_size=batch_size)
test_dataset300 = TestDataset(test,transform_inputsize(300))
test_loader300 = torch.utils.data.DataLoader(test_dataset300, shuffle=False, batch_size=batch_size)

# Load model

In [ ]:
!pip install timm
import timm

In [ ]:
swin_tiny_patch4_window7_224 = timm.create_model('swin_tiny_patch4_window7_224',pretrained=True,num_classes=88,in_chans=3)
swin_tiny_patch4_window7_224.load_state_dict(torch.load('swin_tiny_patch4_window7_224.pt'))

In [ ]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
efficientnetB2 = EfficientNet.from_pretrained('efficientnet-b2', num_classes=88)
efficientnetB2.load_state_dict(torch.load('efficientnet-b2.pt'))

In [ ]:
efficientnetB0 = EfficientNet.from_pretrained('efficientnet-b0', num_classes=88)
efficientnetB0.load_state_dict(torch.load('efficientnet-b0.pt'))

In [ ]:
mixnet_s=timm.create_model('mixnet_s',pretrained=True,num_classes=88,in_chans=3)
mixnet_s.load_state_dict(torch.load('mixnet_s.pt'))

# Inference v1

In [ ]:
model.to(device)
model.eval()
f_pred = []

with torch.no_grad():
    for x,_ in tqdm(test_loader):
        x = x.to(device)
        pred = model(x)
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

# Inference v2 (TTA)

In [ ]:
import ttach as tta

tta_transforms = tta.Compose(
    [
        tta.Rotate90(angles=[0, 180]),
        tta.Multiply(factors=[0.9, 1, 1.1]),
    ]
)

tta_model1 = tta.ClassificationTTAWrapper(swin_tiny_patch4_window7_224, tta_transforms)
tta_model2 = tta.ClassificationTTAWrapper(seresnext50_32x4d, tta_transforms) 
tta_model3 = tta.ClassificationTTAWrapper(efficientnetB2, tta_transforms) 
tta_model4 = tta.ClassificationTTAWrapper(efficientnetB0, tta_transforms) 

In [ ]:
tta_model1.to(device)
tta_model1.eval()
tta_model2.to(device)
tta_model2.eval()
tta_model3.to(device)
tta_model3.eval()
tta_model4.to(device)
tta_model4.eval()


f_pred = []

with torch.no_grad():
    for x,_ in tqdm(test_loader):
        x = x.to(device)
        pred1 = tta_model1(x)
        pred2 = tta_model2(x)
        pred3 = tta_model3(x)
        pred4 = tta_model4(x)
        pred=(pred1+pred2+pred3+pred4)/4
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())


# Inference v3(TTA+difference input size)

In [ ]:
! pip install git+https://github.com/qubvel/ttach

In [ ]:
import ttach

In [ ]:
#ttach.Scale함수는 input output 크기가 달라서 같게 함수 수정
class myScale(ttach.Scale):
    def apply_aug_image(self, image, scale=1, **kwargs):
        image_h,image_w=image.shape[2], image.shape[3]
        if scale != self.identity_param:
            image = ttach.functional.scale(
                image,
                scale,
                interpolation=self.interpolation,
                align_corners=self.align_corners,
            )
            if image.shape[2]>=image_h:
                image=ttach.functional.center_crop(image, image_h, image_w)
            else :
                image=ttach.functional.resize(image, (image_h,image_w))
        return image

In [ ]:
tta_transforms = ttach.Compose(
    [
        ttach.Rotate90(angles=[0, 90, 180, 270]),
        ttach.Multiply(factors=[0.9, 1, 1.1]),
        myScale(scales=[0.9,1, 1.1])
    ]
)

tta_swin_tiny_patch4_window7_224 = ttach.ClassificationTTAWrapper(swin_tiny_patch4_window7_224, tta_transforms)
tta_mixnet_s = ttach.ClassificationTTAWrapper(mixnet_s, tta_transforms) 
tta_efficientnetB2 = ttach.ClassificationTTAWrapper(efficientnetB2, tta_transforms) 
tta_efficientnetB0 = ttach.ClassificationTTAWrapper(efficientnetB0, tta_transforms) 

In [ ]:
tta_swin_tiny_patch4_window7_224.to(device)
tta_swin_tiny_patch4_window7_224.eval()
tta_mixnet_s.to(device)
tta_mixnet_s.eval()
tta_efficientnetB2.to(device)
tta_efficientnetB2.eval()
tta_efficientnetB0.to(device)
tta_efficientnetB0.eval()
f_pred = []

with torch.no_grad():
    for img224,img300 in iter(zip(test_loader224,test_loader300)):

        img224 = img224.to(device)
        img300 = img300.to(device)

        pred1 = tta_swin_tiny_patch4_window7_224(img224)
        pred2 = tta_efficientnetB2(img300)
        pred3 = tta_efficientnetB0(img300)
        pred4 = tta_mixnet_s(img300)

        pred = (pred1+pred2+pred3+pred4)/4
        
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

# Make Submission 

In [ ]:
label_decoder = {val:key for key, val in label_unique_dir.items()}

f_result = [label_decoder[result] for result in f_pred]

In [ ]:
submission = pd.read_csv("./data/sample_submission.csv")

submission["label"] = f_result

submission

In [ ]:
submission.to_csv("swin_tiny_patch4_window7_224+efficientnet-b2+efficientnet-b0+mixnet_s__2.csv", index = False)